In [9]:
import tensorflow as tf

import CNN # External class generated for CNN

import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import time

# Disable eager execution
tf.compat.v1.disable_eager_execution()
import numpy as np
import keras
import seaborn as sns
from matplotlib import pyplot as plt
from scipy.io import loadmat
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import confusion_matrix
from keras.preprocessing.image import ImageDataGenerator
%matplotlib inline
import scipy

epochs=100
batch_size=256


eps_FSGM=0.30

# Select the attacker
#White-box: "FGSM", "BIM", "PGD", 
#Black-box: "SA","ZA"
attack_type = "BIM"
dataset="SVHN"   # MNIST, CIRFAR10

#path='results/MNIST_BA.txt' #this for MNIST images
path='C:/Users/02729/OneDrive - Universiti Teknikal Malaysia Melaka\Desktop/IEEE Access paper/SVHN/SVHN_BIM.txt' 

In [2]:
# Load the data
#https://www.kaggle.com/code/dimitriosroussis/svhn-classification-with-cnn-keras-96-acc

train_raw = loadmat('C:/Users/02729/OneDrive - Universiti Teknikal Malaysia Melaka\Desktop/IEEE Access paper/SVHN/train_32x32.mat')
test_raw = loadmat('C:/Users/02729/OneDrive - Universiti Teknikal Malaysia Melaka\Desktop/IEEE Access paper/SVHN/test_32x32.mat')



In [3]:
# Load images and labels

train_images = np.array(train_raw['X'])
test_images = np.array(test_raw['X'])

train_labels = train_raw['y']
test_labels = test_raw['y']

# Check the shape of the data

print(train_images.shape)
print(test_images.shape)

# Fix the axes of the images

train_images = np.moveaxis(train_images, -1, 0)
test_images = np.moveaxis(test_images, -1, 0)

print(train_images.shape)
print(test_images.shape)

(32, 32, 3, 73257)
(32, 32, 3, 26032)
(73257, 32, 32, 3)
(26032, 32, 32, 3)


In [4]:
'''# Load CIFAR-10 dataset
from tensorflow.keras.datasets import cifar10
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()
unique_classes = np.unique(train_labels)
num_classes = len(unique_classes)
test_labels'''

'# Load CIFAR-10 dataset\nfrom tensorflow.keras.datasets import cifar10\n(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()\nunique_classes = np.unique(train_labels)\nnum_classes = len(unique_classes)\ntest_labels'

In [5]:
'''#Convert train and test images into 'float64' type

train_images = train_images.astype('float64')
test_images = test_images.astype('float64')

# Convert train and test labels into 'int64' type

train_labels = train_labels.astype('int64')
test_labels = test_labels.astype('int64')

# Normalize the images data

print('Min: {}, Max: {}'.format(train_images.min(), train_images.max()))

train_images /= 255.0
test_images /= 255.0'''

"#Convert train and test images into 'float64' type\n\ntrain_images = train_images.astype('float64')\ntest_images = test_images.astype('float64')\n\n# Convert train and test labels into 'int64' type\n\ntrain_labels = train_labels.astype('int64')\ntest_labels = test_labels.astype('int64')\n\n# Normalize the images data\n\nprint('Min: {}, Max: {}'.format(train_images.min(), train_images.max()))\n\ntrain_images /= 255.0\ntest_images /= 255.0"

In [6]:
# Load CIFAR-10 dataset
#from tensorflow.keras.datasets import cifar10
#(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

# Normalize pixel values to [0, 1]
train_images = train_images.astype('float32') / 255.0
test_images = test_images.astype('float32') / 255.0

min_pixel_value = 0.0
max_pixel_value = 1.0

print(train_images.shape, train_labels.shape)

t_consumed=0
classifier, t_consumed = CNN.train(train_images, train_labels, epochs, batch_size,
                                   min_pixel_value, max_pixel_value, test_images, test_labels)
'''# Save the model
classifier.save("/home/app/Desktop/paper 6/code_CIFAR10/classifier.h5")'''

(73257, 32, 32, 3) (73257, 1)
Instructions for updating:
Colocations handled automatically by placer.


C:\Users\02729\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\backend.py:5714: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


Train on 73257 samples, validate on 26032 samples
Epoch 1/10
73257/73257 [==============================] - ETA: 0s - loss: 1.2213 - accuracy: 0.5928

C:\Users\02729\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training_v1.py:2335: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


73257/73257 [==============================] - 260s 4ms/sample - loss: 1.2213 - accuracy: 0.5928 - val_loss: 0.4498 - val_accuracy: 0.8662
Epoch 2/10
73257/73257 [==============================] - 250s 3ms/sample - loss: 0.4633 - accuracy: 0.8665 - val_loss: 0.3772 - val_accuracy: 0.8942
Epoch 3/10
73257/73257 [==============================] - 236s 3ms/sample - loss: 0.3668 - accuracy: 0.8988 - val_loss: 0.2828 - val_accuracy: 0.9237
Epoch 4/10
73257/73257 [==============================] - 216s 3ms/sample - loss: 0.3075 - accuracy: 0.9167 - val_loss: 0.2476 - val_accuracy: 0.9342
Epoch 5/10
73257/73257 [==============================] - 216s 3ms/sample - loss: 0.2755 - accuracy: 0.9253 - val_loss: 0.2631 - val_accuracy: 0.9271
Epoch 6/10
73257/73257 [==============================] - 215s 3ms/sample - loss: 0.2452 - accuracy: 0.9336 - val_loss: 0.2302 - val_accuracy: 0.9400
Epoch 7/10
73257/73257 [==============================] - 296s 4ms/sample - loss: 0.2280 - accuracy: 0.9391 - v

'# Save the model\nclassifier.save("/home/app/Desktop/paper 6/code_CIFAR10/classifier.h5")'

In [10]:
for i in range(30, 40, 10):
    eps_FSGM= (i/100)
    print("..............EPS..........: ", eps_FSGM)
    #Evaluate the model
    acc, prc, rec, f1 = CNN.test(classifier, test_images, test_labels)

    #write the results
    import Writer
    results ="===========\n" +  dataset + "-Regular:\n acc, prc, rec, f1, epochs, batch size, time: acc, prc, rec, f1\n"
    results +=str(acc) + "," + str(prc) + "," + str(rec)+ "," + str(f1) + "," + str(epochs)+ "," + str(batch_size) + "," + str(int(t_consumed)) +"\n" 
    Writer.write_results(results, path)
    #==================================
    
    # Extract the features for the regular images
    from tensorflow.keras import models

    model_output = models.Model(inputs=classifier._model.input, outputs=classifier._model.get_layer('dense_features').output)
    Features_regular = model_output.predict(test_images)
    print(Features_regular.shape)   
    #==================================

    # Generate adversarial examples using the FastGradientMethod attack
    from art.attacks.evasion import FastGradientMethod, BasicIterativeMethod, ProjectedGradientDescent #white-box
    from art.attacks.evasion import ZooAttack, SquareAttack, BoundaryAttack #black-box


    # Initialize the attack
    attack = None

    # Select the attack
    if attack_type == "FGSM":
        attack = FastGradientMethod(estimator=classifier, eps=eps_FSGM)
    if attack_type == "BIM":
        attack = BasicIterativeMethod(estimator=classifier, eps=eps_FSGM)
    elif attack_type == "PGD":
        attack = ProjectedGradientDescent(estimator=classifier, eps=eps_FSGM)
    elif attack_type == "SA":
        attack = SquareAttack(estimator=classifier,eps=eps_FSGM)
    elif attack_type == "ZA":
        attack = ZooAttack(classifier=classifier, max_iter=2, learning_rate=eps_FSGM)

    adv_images = attack.generate(test_images)
    #=============================================
    
    # Evaluate the model on the adversarial test set images
    acc, prc, rec, f1 = CNN.test(classifier, adv_images, test_labels)

    #write the results
    results =dataset + "-Adversarial:\n acc, prc, rec, f1, epochs, attack_type, eps_FSGM\n"
    results +=str(acc) + "," + str(prc) + "," + str(rec)+ "," + str(f1) + "," + attack_type + "," + str(eps_FSGM) +"\n" 
    Writer.write_results(results, path)
    #=============================================

    # Extract the features for the adversarial images
    model_output = models.Model(inputs=classifier._model.input, outputs=classifier._model.get_layer('dense_features').output)
    Features_adversarial = model_output.predict(adv_images)
    print(Features_adversarial.shape)
    #=============================================

    # merge regular and feature test images
    from sklearn.model_selection import train_test_split

    #merge data features of regular and adversarial images
    reg_adv_images = np.concatenate((Features_regular, Features_adversarial), axis=0)

    # generate labels for regular and adversarial, and merge them

    label_reg = np.zeros((int(Features_regular.shape[0]), 1))
    label_adv = np.ones((int(Features_adversarial.shape[0]), 1))
    reg_adv_labels=np.concatenate((label_reg, label_adv), axis=0)

    print("data details: ", reg_adv_images.shape, " , labels: ", reg_adv_labels.shape)

    X_train, X_test, y_train, y_test = train_test_split(reg_adv_images, reg_adv_labels, test_size=0.20, random_state=42)

    #fractin to use a part of the merged regular and adversarial images
    fraction=1
    #reduce the length because the memory is out
    X_train=X_train[:int(X_train.shape[0]/fraction)]
    X_test=X_test[:(int(X_test.shape[0]/fraction))]
    y_train=y_train[:(int(y_train.shape[0]/fraction))]
    y_test=y_test[:(int(y_test.shape[0]/fraction))]
    X_train, X_test, y_train, y_test
    print("data after split : X_train, X_test, y_train, y_test\n", X_train.shape, X_test.shape, y_train.shape, y_test.shape)

    #=============================================

    # integrate the adversarial detector model to CNN to exclude adversarial images or outlier images.

    size_advs=10000
    test_reg_adv_images=np.concatenate((test_images,adv_images[:size_advs]), axis=0)
    test_reg_adv_labels=np.concatenate((test_labels,test_labels[:size_advs]), axis=0)
    print(test_reg_adv_images.shape, test_reg_adv_labels.shape)

    # Shuffle the data
    indices = np.arange(len(test_reg_adv_images))
    np.random.shuffle(indices)
    test_reg_adv_images= test_reg_adv_images[indices]
    test_reg_adv_labels = test_reg_adv_labels[indices]

    # Evaluate the model after integrate regular and adversarial images
    acc, prc, rec, f1 = CNN.test(classifier, test_reg_adv_images, test_reg_adv_labels)
    results =dataset + "-Regular+Adversarial:\n acc, prc, rec, f1\n"
    results +=str(acc) + "," + str(prc) + "," + str(rec)+ "," + str(f1) +"\n" 
    Writer.write_results(results, path)

    #=============================================

    # Extract features 
    model_output = models.Model(inputs=classifier._model.input, outputs=classifier._model.get_layer('dense_features').output)
    Features_adv_reg = model_output.predict(test_reg_adv_images)
    print(Features_adversarial.shape)
    #=============================================

    #to ensure that the size of regular and adversarial image size are the same to do fair evaluation
    def make_size_fit(clean_images,clean_labels, attacked_images, attacked_labels):
        diff=int(np.absolute(clean_images.shape[0] - attacked_images.shape[0])/2)
        size=int((clean_images.shape[0]+attacked_images.shape[0])/2)
        temp_clean_images=np.ones((size,32,32,3))
        temp_attacked_images=np.ones((size,32,32,3))

        temp_clean_labels=np.ones((size,1))
        temp_attacked_labels=np.ones((size,1))

        if(clean_images.shape[0]>attacked_images.shape[0]):
            temp_clean_images=clean_images[:size]
            temp_clean_labels=clean_labels[:size]

            temp_attacked_images=np.concatenate((attacked_images,clean_images[-diff:]), axis=0)
            temp_attacked_labels=np.concatenate((temp_attacked_labels,clean_labels[-diff:]), axis=0)

        elif(clean_images.shape[0]<attacked_images.shape[0]):
            temp_attacked_images=attacked_images[:size]
            temp_attacked_labels=attacked_labels[:size]

            temp_clean_images=np.concatenate((clean_images,attacked_images[-diff:]), axis=0)
            temp_clean_labels=np.concatenate((clean_labels,attacked_labels[-diff:]), axis=0)

        clean_images=temp_clean_images
        clean_labels=temp_clean_labels

        attacked_images=temp_attacked_images
        attacked_labels=temp_attacked_labels

        #print("clean_images.shape, attacked_images.shape", clean_images.shape, attacked_images.shape)
        #print("clean_labels.shape, attacked_labels.shape", clean_labels.shape, attacked_labels.shape)

        return clean_images,clean_labels, attacked_images, attacked_labels

    #Exclude adversarial images from testing and use only the regular images

    def retrurn_clean_images(adv_images_pred_binary):

        # Count the number of times 1 appears in the array
        number_of_1s = np.count_nonzero(adv_images_pred_binary == 1)
        # Count the number of times 0 appears in the array
        number_of_0s = np.count_nonzero(adv_images_pred_binary == 0)
        print("The number of attacked immages 1 is:", number_of_1s)
        print("The number of clean images 0 is:", number_of_0s)

        #Exclude adversarial images from testing and use only the regular images
        # Create a boolean mask use only the clean images
        mask_clean = np.where(adv_images_pred_binary == 0)[0]
        mask_adv = np.where(adv_images_pred_binary == 1)[0]

        # Exclude adversarial images
        #clean images
        clean_images = test_reg_adv_images[mask_clean]
        clean_labels = test_reg_adv_labels[mask_clean]
        #Adversarial images,attacked images
        attacked_images = test_reg_adv_images[mask_adv]
        attacked_labels = test_reg_adv_labels[mask_adv]

        clean_images,clean_labels, attacked_images, attacked_labels = make_size_fit(clean_images,clean_labels, attacked_images, attacked_labels)

        print("clean_images:", clean_images.shape)
        print("clean_labels:", clean_labels.shape)

        return clean_images, clean_labels


    def eval_model_after_exclude_adv_image(model_name, new_test_clean_images, new_test_clean_labels, t_consumed):# Evaluate the model after excluding the adversarial images
        acc, prc, rec, f1 = CNN.test(classifier, new_test_clean_images, new_test_clean_labels)
        results =model_name + ":\n acc, prc, rec, f1, time_detection\n"
        results +=str(acc) + "," + str(prc) + "," + str(rec)+ "," + str(f1) + "," + str(int(t_consumed)) + "\n" 
        Writer.write_results(results, path)
    #=============================================

    #Evaluate the mosdel after inegrate NN as outlier detection to exclude Adversarial images
    import ML_Classifiers as MLC
    t_str=time.time()

    adv_images_pred_binary= MLC.NN(X_train, X_test, y_train, y_test,eps_FSGM, attack_type, Features_adv_reg)
    new_test_clean_images, new_test_clean_labels = retrurn_clean_images(adv_images_pred_binary)

    t_consumed=time.time()-t_str
    eval_model_after_exclude_adv_image("CNN+NN_outlier_detection", new_test_clean_images, new_test_clean_labels, t_consumed)
    #=============================================

    ##Evaluate the mosdel after inegrate RF as outlier detection to exclude Adversarial images
    import ML_Classifiers as MLC
    t_str=time.time()

    adv_images_pred_binary= MLC.Random_Forest(X_train, X_test, y_train, y_test,Features_adv_reg)
    new_test_clean_images, new_test_clean_labels = retrurn_clean_images(adv_images_pred_binary)

    t_consumed=time.time()-t_str
    eval_model_after_exclude_adv_image("CNN+RF_outlier_detection", new_test_clean_images, new_test_clean_labels,t_consumed)

    #=============================================

    ##Evaluate the mosdel after inegrate DT as outlier detection to exclude Adversarial images
    t_str=time.time()
    adv_images_pred_binary= MLC.DT(X_train, X_test, y_train, y_test,Features_adv_reg)
    new_test_clean_images, new_test_clean_labels = retrurn_clean_images(adv_images_pred_binary)

    t_consumed=time.time()-t_str
    eval_model_after_exclude_adv_image("CNN+DT_outlier_detection", new_test_clean_images, new_test_clean_labels,t_consumed)
    #=============================================

    ##Evaluate the mosdel after inegrate KNN as outlier detection to exclude Adversarial images
    t_str=time.time()
    adv_images_pred_binary= MLC.KNN(X_train, X_test, y_train, y_test,Features_adv_reg)
    new_test_clean_images, new_test_clean_labels = retrurn_clean_images(adv_images_pred_binary)

    t_consumed=time.time()-t_str
    eval_model_after_exclude_adv_image("CNN+KNN_outlier_detection", new_test_clean_images, new_test_clean_labels,t_consumed)
    #=============================================

    ##Evaluate the mosdel after inegrate GBM as outlier detection to exclude Adversarial images
    t_str=time.time()

    adv_images_pred_binary= MLC.GBM(X_train, X_test, y_train, y_test,Features_adv_reg)
    new_test_clean_images, new_test_clean_labels = retrurn_clean_images(adv_images_pred_binary)

    t_consumed=time.time()-t_str
    eval_model_after_exclude_adv_image("CNN+GBM_outlier_detection", new_test_clean_images, new_test_clean_labels, t_consumed)
    #=============================================

    ##Evaluate the mosdel after inegrate XGB as outlier detection to exclude Adversarial images
    t_str=time.time()

    adv_images_pred_binary= MLC.XGB(X_train, X_test, y_train, y_test,Features_adv_reg)

    new_test_clean_images, new_test_clean_labels = retrurn_clean_images(adv_images_pred_binary)

    t_consumed=time.time()-t_str
    eval_model_after_exclude_adv_image("CNN+XGB_outlier_detection", new_test_clean_images, new_test_clean_labels, t_consumed)
    #=============================================

..............EPS..........:  0.3
acc, prc, rec, f1: 94.14 94.16 94.14 94.14


C:\Users\02729\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


(26032, 32)


PGD - Random Initializations:   0%|          | 0/1 [00:00<?, ?it/s]

PGD - Iterations:   0%|          | 0/100 [00:00<?, ?it/s]

...

GBM detection: acc, prc, rec, f1: 99.86 99.86 99.86 99.86
The number of attacked immages 1 is: 10006
The number of clean images 0 is: 26026


C:\Users\02729\OneDrive - Universiti Teknikal Malaysia Melaka\Desktop\IEEE Access paper\SVHN\ML_Classifiers.py:200: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if(adv_reg_images!=""):


clean_images: (18016, 32, 32, 3)
clean_labels: (18016, 1)
acc, prc, rec, f1: 93.93 93.96 93.93 93.94
XGB detection: acc, prc, rec, f1: 98.9 98.9 98.9 98.9
The number of attacked immages 1 is: 10080
The number of clean images 0 is: 25952


C:\Users\02729\OneDrive - Universiti Teknikal Malaysia Melaka\Desktop\IEEE Access paper\SVHN\ML_Classifiers.py:161: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if(adv_reg_images!=""):


clean_images: (18016, 32, 32, 3)
clean_labels: (18016, 1)
acc, prc, rec, f1: 93.52 93.59 93.52 93.54


In [8]:
##Evaluate the mosdel after inegrate FAM as outlier detection to exclude Adversarial images

for i in range(30, 40, 10):
    eps_FSGM= (i/100)
    print("..............EPS..........: ", eps_FSGM)

    import FAM 

    t_str=time.time()
    ArtMap=FAM.train(X_train[:5000], y_train[:5000])

    adv_images_pred_binary = FAM.test(X_test[:1000], y_test[:1000], ArtMap, Features_adv_reg)

    adv_images_pred_binary=np.array(adv_images_pred_binary)
    new_test_clean_images, new_test_clean_labels = retrurn_clean_images(adv_images_pred_binary)

    t_consumed=time.time()-t_str
    eval_model_after_exclude_adv_image("CNN+FAM_outlier_detection", new_test_clean_images, new_test_clean_labels, t_consumed)




..............EPS..........:  0.05
142  values were replaced from  1000
FAM detection: acc, prc, rec, f1: 19.7 8.73 6.72 7.59


C:\Users\02729\OneDrive - Universiti Teknikal Malaysia Melaka\Desktop\IEEE Access paper\SVHN\FAM.py:72: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if(adv_reg_images!=""):


{'index': 1, 'ArtB': [0.0, 1.0], 'id': '0010'}
The number of attacked immages 1 is: 20384
The number of clean images 0 is: 15648
clean_images: (18016, 32, 32, 3)
clean_labels: (18016, 1)
acc, prc, rec, f1: 48.16 53.88 48.16 48.82
..............EPS..........:  0.1
142  values were replaced from  1000
FAM detection: acc, prc, rec, f1: 19.7 8.73 6.72 7.59


C:\Users\02729\OneDrive - Universiti Teknikal Malaysia Melaka\Desktop\IEEE Access paper\SVHN\FAM.py:72: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if(adv_reg_images!=""):


{'index': 1, 'ArtB': [0.0, 1.0], 'id': '0010'}
The number of attacked immages 1 is: 20384
The number of clean images 0 is: 15648
clean_images: (18016, 32, 32, 3)
clean_labels: (18016, 1)


KeyboardInterrupt: 